In [1]:
import statistics
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Import datasets
trainData = pd.read_csv("../data/train/train.csv")
sentimentScore = pd.read_csv("../data/train/sentimentScores.csv")

In [3]:
# Count words in descirptions for analysis
sentences = trainData['Description'].tolist()
counts = []
for sent in sentences:
    sent = str(sent)
    numWords = len(str.split(sent))
    counts.append(numWords)
trainData['DescriptionLength'] = counts

In [4]:
# Get mean sentiment score
pets = trainData['PetID'].tolist()
means = []
for pet in pets:
    scores = sentimentScore.loc[sentimentScore['PetID'] == pet]
    # meanMag = statistics.mean(scores['SentimentMagnitude'])
    meanScore = statistics.mean(scores['SentimentScore'])
    means.append(meanScore)
trainData['MeanSentimentScore'] = means

In [5]:
breedLabels = pd.read_csv("../data/breed_labels.csv")
stateLabels = pd.read_csv("../data/state_labels.csv")
colorLabels = pd.read_csv("../data/color_labels.csv")

In [6]:
trainData = trainData.merge(breedLabels, how = "left",
                            left_on = ["Type", "Breed1"],
                            right_on = ["Type", "BreedID"])
trainData = trainData.merge(breedLabels, how = "left",
                            left_on = ["Type", "Breed2"],
                            right_on = ["Type", "BreedID"])
trainData = trainData.merge(colorLabels, how = "left",
                            left_on = "Color1",
                            right_on = "ColorID")
trainData = trainData.merge(colorLabels, how = "left",
                            left_on = "Color2",
                            right_on = "ColorID")
trainData = trainData.merge(colorLabels, how = "left",
                            left_on = "Color3",
                            right_on = "ColorID")
trainData = trainData.merge(stateLabels, how = "left",
                            left_on = "State",
                            right_on = "StateID")
trainData = trainData.drop(["BreedID_x", "BreedID_y", "ColorID_x", "ColorID_y", "ColorID", "StateID"], axis = 1)
trainData = trainData.rename(columns = {'BreedName_x': 'BreedName1',
                                        'BreedName_y': 'BreedName2',
                                        'ColorName_x': 'ColorName1',
                                        'ColorName_y': 'ColorName2',
                                        'ColorName': 'ColorName3'})

In [8]:
# Send to sqlite
engine = create_engine('sqlite:///../db/database.sqlite')
trainData.to_sql('Pet_Training_Data', con = engine, index = False)

In [9]:
trainData.dtypes

Type                    int64
Name                   object
Age                     int64
Breed1                  int64
Breed2                  int64
Gender                  int64
Color1                  int64
Color2                  int64
Color3                  int64
MaturitySize            int64
FurLength               int64
Vaccinated              int64
Dewormed                int64
Sterilized              int64
Health                  int64
Quantity                int64
Fee                     int64
State                   int64
RescuerID              object
VideoAmt                int64
Description            object
PetID                  object
PhotoAmt              float64
AdoptionSpeed           int64
DescriptionLength       int64
MeanSentimentScore    float64
BreedName1             object
BreedName2             object
ColorName1             object
ColorName2             object
ColorName3             object
StateName              object
dtype: object